In [52]:
import sagemaker
import boto3
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Initialize SageMaker and S3 clients
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
s3_client = boto3.client('s3')

# Download Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)


# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

#y = encoder.fit_transform(y_train.reshape(-1, 1))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.astype(int)
y_test = y_test.astype(int)



/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [53]:
# Save the data as .npy files
np.save('/tmp/X_train.npy', X_train)
np.save('/tmp/y_train.npy', y_train)
np.save('/tmp/X_test.npy', X_test)
np.save('/tmp/y_test.npy', y_test)

bucket = 'anomaly-model5'

# Upload the data to S3
train_input_path = sagemaker_session.upload_data('/tmp/X_train.npy', bucket=bucket, key_prefix='iris/data')
train_output_path = sagemaker_session.upload_data('/tmp/y_train.npy', bucket=bucket, key_prefix='iris/data')
test_input_path = sagemaker_session.upload_data('/tmp/X_test.npy', bucket=bucket, key_prefix='iris/data')
test_output_path = sagemaker_session.upload_data('/tmp/y_test.npy', bucket=bucket, key_prefix='iris/data')


In [57]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(120, 4)
(120, 3)
(30, 4)
(30, 3)


In [55]:
train_input_path = 's3://anomaly-model5/iris/data/'
test_input_path = 's3://anomaly-model5/iris/data/'

In [49]:
train_input_path

's3://anomaly-model5/iris/data/'

In [62]:
from sagemaker.tensorflow import TensorFlow

iris_estimator = TensorFlow(entry_point='train3.py',
                          role=role,
                          framework_version='2.13',
                          py_version='py310',
                          train_instance_count=1,
                          train_instance_type='ml.g4dn.xlarge',
                          hyperparameters={'learning_rate': 0.001, 'epochs': 100},
                           output_path='s3://anomaly-model5/iris/model'
                           )
                

iris_estimator.fit({'train': train_input_path, 'test': test_input_path})

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-01-28-16-11-52-553


2024-01-28 16:11:52 Starting - Starting the training job......
2024-01-28 16:12:23 Starting - Preparing the instances for training......
2024-01-28 16:13:31 Downloading - Downloading input data...
2024-01-28 16:14:11 Downloading - Downloading the training image............
2024-01-28 16:16:07 Training - Training image download completed. Training in progress..2024-01-28 16:16:15.577280: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 16:16:15.629567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 16:16:18,1

In [64]:
predictor = iris_estimator.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')


INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: tensorflow-training-2024-01-28-16-17-28-935
INFO:sagemaker:Creating endpoint-config with name tensorflow-training-2024-01-28-16-17-28-935
INFO:sagemaker:Creating endpoint with name tensorflow-training-2024-01-28-16-17-28-935


-------!

In [68]:
predictor.predict(X_test)

{'predictions': [[0.00165985862, 0.63594377, 0.362396419],
  [0.997208297, 0.00275599118, 3.56601959e-05],
  [2.7627118e-07, 0.0408964455, 0.959103286],
  [0.00173521321, 0.598545611, 0.399719238],
  [0.00176696235, 0.725758791, 0.272474259],
  [0.995565593, 0.00435964, 7.47212325e-05],
  [0.0307704676, 0.845696211, 0.123533286],
  [0.000134844435, 0.323551565, 0.676313639],
  [0.000740162388, 0.497689486, 0.501570284],
  [0.0132906968, 0.825321674, 0.161387682],
  [0.000188961538, 0.338153273, 0.661657751],
  [0.993761361, 0.0060966881, 0.000141917044],
  [0.997817159, 0.00215134327, 3.15455545e-05],
  [0.993932784, 0.00593905756, 0.000128026732],
  [0.996561, 0.00337596, 6.30252907e-05],
  [0.00188956968, 0.65293473, 0.345175743],
  [5.42074395e-06, 0.0839177072, 0.916076839],
  [0.00983500481, 0.784931183, 0.205233812],
  [0.00158443477, 0.558496594, 0.439919],
  [6.97981e-06, 0.0869399905, 0.913053036],
  [0.991080225, 0.00870367885, 0.00021608136],
  [0.000258905377, 0.337558627, 

In [69]:
y_test

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

In [70]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: tensorflow-training-2024-01-28-16-17-28-935
INFO:sagemaker:Deleting endpoint with name: tensorflow-training-2024-01-28-16-17-28-935


In [180]:
#model is stored here:
model_data = 's3://anomaly-model5/iris/model/tensorflow-training-2024-01-28-16-11-52-553/output/model.tar.gz'

tf_model = TensorFlowModel(model_data=model_data,
                           role=role,
                           framework_version='2.13',  # Replace with the version you used
                           sagemaker_session=sagemaker_session,
                           #entry_point = 'predict.py'
                           #source_dir="model_directory"
                          )
# Deploy the model to an endpoint
predictor = tf_model.deploy(initial_instance_count=1,
                            instance_type='ml.g4dn.xlarge')

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: tensorflow-inference-2024-01-29-23-34-13-743
INFO:sagemaker:Creating endpoint-config with name tensorflow-inference-2024-01-29-23-34-14-356
INFO:sagemaker:Creating endpoint with name tensorflow-inference-2024-01-29-23-34-14-356


-------!

In [229]:
predictor.endpoint

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'tensorflow-inference-2024-01-30-01-45-49-702'

In [188]:
predictor.predict(X_test)

{'predictions': [[0.00165985862, 0.63594377, 0.362396419],
  [0.997208297, 0.00275599118, 3.56601959e-05],
  [2.7627118e-07, 0.0408964455, 0.959103286],
  [0.00173521321, 0.598545611, 0.399719238],
  [0.00176696235, 0.725758791, 0.272474259],
  [0.995565593, 0.00435964, 7.47212325e-05],
  [0.0307704676, 0.845696211, 0.123533286],
  [0.000134844435, 0.323551565, 0.676313639],
  [0.000740162388, 0.497689486, 0.501570284],
  [0.0132906968, 0.825321674, 0.161387682],
  [0.000188961538, 0.338153273, 0.661657751],
  [0.993761361, 0.0060966881, 0.000141917044],
  [0.997817159, 0.00215134327, 3.15455545e-05],
  [0.993932784, 0.00593905756, 0.000128026732],
  [0.996561, 0.00337596, 6.30252907e-05],
  [0.00188956968, 0.65293473, 0.345175743],
  [5.42074395e-06, 0.0839177072, 0.916076839],
  [0.00983500481, 0.784931183, 0.205233812],
  [0.00158443477, 0.558496594, 0.439919],
  [6.97981e-06, 0.0869399905, 0.913053036],
  [0.991080225, 0.00870367885, 0.00021608136],
  [0.000258905377, 0.337558627, 

In [234]:
predictor.predict([.2,'xd',3,4])

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{    "error": "Failed to process element: 0 of 'instances' list. Error: INVALID_ARGUMENT: JSON Value: \"xd\" Type: String is not of expected type: float"}". See https://eu-north-1.console.aws.amazon.com/cloudwatch/home?region=eu-north-1#logEventViewer:group=/aws/sagemaker/Endpoints/tensorflow-inference-2024-01-30-01-45-49-702 in account 700383664770 for more information.

In [225]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: tensorflow-inference-2024-01-30-01-38-28-721
INFO:sagemaker:Deleting endpoint with name: tensorflow-inference-2024-01-30-01-38-28-721


In [190]:
import boto3

# Create a CloudWatch Logs client
log_client = boto3.client('logs')

# Define your log group name (for SageMaker, it typically starts with '/aws/sagemaker/Endpoints')
log_group_name = '/aws/sagemaker/Endpoints/tensorflow-inference-2024-01-29-23-41-50-377'

# List the log streams (optional, helps to find the relevant stream)
streams = log_client.describe_log_streams(logGroupName=log_group_name)
for stream in streams['logStreams']:
    print(stream['logStreamName'])



AllTraffic/i-0b6f80222fed6de8d


In [191]:
# Get log events from a specific log stream
log_stream_name = 'AllTraffic/i-0b6f80222fed6de8d'  # Replace with your log stream name
events = log_client.get_log_events(logGroupName=log_group_name, logStreamName=log_stream_name)

# Print log events
for event in events['events']:
    print(event['message'])


7 2024-01-29 23:44:48,094 INFO     PYTHON SERVICE: False
7 2024-01-29 23:44:48,094 INFO     starting services
7 2024-01-29 23:44:48,095 INFO     using default model name: model
7 2024-01-29 23:44:48,095 INFO     tensorflow serving model config: 
model_config_list: {
  config: {
    name: 'model'
    base_path: '/opt/ml/model'
    model_platform: 'tensorflow'
    model_version_policy: {
      specific: {
        versions: 1
      }
    }
  }
}
7 2024-01-29 23:44:51,396 INFO     tensorflow version info:
2024-01-29 23:44:48.166681: I external/org_tensorflow/tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
TensorFlow ModelServer: 2.13.0-rc2+dev.sha.no_git
TensorFlow Library: 2.13.0
7 2024-01-29 23:44:51,396 INFO     tensorflow serving command: tensorflow_model_server --port=2500

In [187]:
predictor.predict('xd')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{    "error": "Failed to process element: 0 of 'instances' list. Error: INVALID_ARGUMENT: JSON Value: \"xd\" Type: String is not of expected type: float"}". See https://eu-north-1.console.aws.amazon.com/cloudwatch/home?region=eu-north-1#logEventViewer:group=/aws/sagemaker/Endpoints/tensorflow-inference-2024-01-29-23-41-50-377 in account 700383664770 for more information.

In [79]:
import numpy as np
test = np.load('input_notsequenced.npy')
sz = test.shape[0] - 10 + 1
sequences = np.zeros((sz, 10, 256, 256, 1))
# apply the sliding window technique to get the sequences
for i in range(0, sz):
    clip = np.zeros((10, 256, 256, 1))
    for j in range(0, 10):
        clip[j] = test[i + j, :, :, :]
    sequences[i] = clip
np.save('sequences.npy', sequences)

In [81]:
train_IO = sagemaker_session.upload_data('./sequences.npy', bucket=bucket, key_prefix='iris2/data')
train_IO

's3://anomaly-model5/iris2/data/sequences.npy'

In [91]:
train_IO = 's3://anomaly-model5/iris3'

from sagemaker.tensorflow import TensorFlow

anom_estimator = TensorFlow(entry_point='train4.py',
                          role=role,
                          framework_version='2.13',
                          py_version='py310',
                          train_instance_count=1,
                          train_instance_type='ml.g4dn.2xlarge',
                          hyperparameters={'learning_rate': 0.01, 'epochs': 1},
                           output_path='s3://anomaly-model5/iris6/model'
                           )
                

anom_estimator.fit({'train': train_IO})

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-01-29-21-07-41-619


2024-01-29 21:07:41 Starting - Starting the training job.........
2024-01-29 21:08:57 Starting - Preparing the instances for training......
2024-01-29 21:10:04 Downloading - Downloading input data.....................
2024-01-29 21:13:35 Training - Training image download completed. Training in progress..2024-01-29 21:13:38.405319: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 21:13:38.456405: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 21:13:40,770 sagemaker-training-toolkit INFO     Imported framewor

In [5]:
model_data2 = 's3://anomaly-model5/iris4/model/tensorflow-training-2024-01-29-21-07-41-619/output/model.tar.gz'


import sagemaker
from sagemaker.tensorflow import TensorFlowModel

# Set up the SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()  # Replace with your IAM role ARN if not running on SageMaker notebook

# Define model data (S3 path where the model artifact is saved)
#model_data = 's3://anomaly-model5/iris/model/tensorflow-training-2024-01-28-16-11-52-553/output/model.tar.gz'

# Create a TensorFlow model
tf_model = TensorFlowModel(model_data=model_data2,
                           role=role,
                           framework_version='2.13',  # Replace with the version you used
                           sagemaker_session=sagemaker_session,
                           #entry_point = 'predict.py',
                           # source_dir="model_directory"
                          )

# Deploy the model to an endpoint
predictor2 = tf_model.deploy(initial_instance_count=1,
                            instance_type='ml.g4dn.2xlarge')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
-------!

In [ ]:
predictor2.endpoint

In [6]:
test = np.load('sequences.npy')
test.shape

(191, 10, 256, 256, 1)

In [7]:
import numpy as np

In [8]:
test = test[0:1]
test.shape

(1, 10, 256, 256, 1)

In [12]:
predictor2.predict([.1,.2])

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{    "error": "2 root error(s) found.\n  (0) INVALID_ARGUMENT: Input to reshape is a tensor with 2 values, but the requested shape requires a multiple of 65536\n\t [[{{function_node __inference__wrapped_model_15235}}{{node sequential/time_distributed/Reshape}}]]\n\t [[StatefulPartitionedCall/_223]]\n  (1) INVALID_ARGUMENT: Input to reshape is a tensor with 2 values, but the requested shape requires a multiple of 65536\n\t [[{{function_node __inference__wrapped_model_15235}}{{node sequential/time_distributed/Reshape}}]]\n0 successful operations.\n0 derived errors ignored."}". See https://eu-north-1.console.aws.amazon.com/cloudwatch/home?region=eu-north-1#logEventViewer:group=/aws/sagemaker/Endpoints/tensorflow-inference-2024-01-30-07-35-40-580 in account 700383664770 for more information.

In [ ]:
#test = test[0:10]
predictor2.predict(test)